In [1]:
###### DO NOT REMOVE THIS BLOCK ######

import os
import sys

if 'VIRTUAL_ENV' in os.environ:
    py_version = sys.version_info[:2] # formatted as X.Y 
    py_infix = os.path.join('lib', ('python%d.%d' % py_version))
    virtual_site = os.path.join(os.environ.get('VIRTUAL_ENV'), py_infix, 'site-packages')
    dist_site = os.path.join('/usr', py_infix, 'dist-packages')

    # OPTIONAL: exclude debian-based system distributions sites
    # ADD1: sys.path must be a list
    sys.path = list(filter(lambda p: not p.startswith(dist_site), sys.path))

    # add virtualenv site
    # ADD2: insert(0 is wrong and breaks conformance of sys.path
    sys.path.insert(1, virtual_site)

In [2]:
import utils
import json
import datetime
import numpy
from textblob import TextBlob

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
        .master("local") \
        .appName("Yelp Shut Down Prediction") \
        .config("k1", "v1") \
        .getOrCreate()

In [4]:
from pyspark.sql import Row

In [5]:
user = utils.load("dicts/user.p")
store = utils.load("dicts/store.p")
reviews = utils.load("dicts/reviews.p")
store_review = utils.load("dicts/store_review.p")
store_user = utils.load("dicts/store_user.p")

test_id = utils.load("data_id/test.p")

In [6]:
def getPosNeg_score(b_id):
    pos = 0.0
    neg = 0.0
    pos_len = 0.0
    neg_len = 0.0

    for review_id in store_review[b_id]:
        if reviews[review_id]["pol"] >= 0:
            pos += reviews[review_id]["pol"]
            pos_len += 1
        else:
            neg += reviews[review_id]["pol"]
            neg_len += 1

    if pos_len:
        pos /= pos_len
    if neg_len:
        neg /= neg_len
    return pos, neg

In [7]:
def get_y(b_id, observe_t = 12):
    life_time = store[b_id]["end_t"] - store[b_id]["start_t"]
    longer = life_time.days/30 - observe_t
    if longer > 0:
        return store[b_id]["is_open"]
    else:
        if store[b_id]["is_open"]:
            return -1
        else:
            return 0

In [8]:
def elite_user(b_id, observe_t=12):
    cnt = 0
    for user_id in store_user[b_id]:
        if user[user_id]["elite"][0] == "None":
            continue
        for yr in user[user_id]["elite"]:
            eli_date = datetime.datetime(int(yr), 1, 1)
            diff = eli_date - store[b_id]["start_t"]
            if diff.days/30 < observe_t:
                cnt = cnt + 1
                break
    return cnt

In [9]:
def sentimentAnalizer(text):
    pattern = TextBlob(text)
    return pattern.sentiment[0]

In [10]:
import os
memory = '20g'
pyspark_submit_args = ' --driver-memory ' + memory + ' pyspark-shell'
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

In [11]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType, FloatType, StringType


city = udf(lambda b_id: store[b_id]["city"], StringType())
state = udf(lambda b_id: store[b_id]["state"], StringType())
stars = udf(lambda b_id: store[b_id]["stars"], FloatType())
popularity = udf(lambda b_id: len(store_review[b_id]), IntegerType())
name_size = udf(lambda b_id: len(store[b_id]["name"].split()), IntegerType())
name_polar = udf(lambda b_id: sentimentAnalizer(store[b_id]["name"]), FloatType())
# pos_neg_score = udf(lambda b_id: getPosNeg_score(b_id), FloatType())
elite_cnt = udf(lambda b_id: elite_user(b_id), IntegerType())
get_label = udf(lambda b_id: get_y(b_id), IntegerType())

In [12]:
train_id = utils.load("data_id/train.p")
train_id = [[idx] for idx in train_id]

data_f = spark.createDataFrame(train_id, ['biz_id'])

In [13]:
data_f = data_f.withColumn("city", city(data_f['biz_id']))
data_f = data_f.withColumn("state", state(data_f['biz_id']))
data_f = data_f.withColumn("stars", stars(data_f['biz_id']))
data_f = data_f.withColumn("popularity", popularity(data_f['biz_id']))
data_f = data_f.withColumn("name_size", name_size(data_f['biz_id']))
data_f = data_f.withColumn("name_polar", name_polar(data_f['biz_id']))
# data_f = data_f.withColumn("pos_neg_score", pos_neg_score(data_f['biz_id']))
data_f = data_f.withColumn("elite_cnt", elite_cnt(data_f['biz_id']))
data_f = data_f.withColumn("y", get_label(data_f['biz_id']))
data_f.show(5)

Py4JJavaError: An error occurred while calling o212.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/srv/spark/python/lib/pyspark.zip/pyspark/worker.py", line 161, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile)
  File "/srv/spark/python/lib/pyspark.zip/pyspark/worker.py", line 97, in read_udfs
    arg_offsets, udf = read_single_udf(pickleSer, infile)
  File "/srv/spark/python/lib/pyspark.zip/pyspark/worker.py", line 78, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
  File "/srv/spark/python/lib/pyspark.zip/pyspark/worker.py", line 56, in read_command
    command = serializer.loads(command.value)
  File "/srv/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 431, in loads
    re

In [ ]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

categorical = ["city", "state"]
stringIndexer = StringIndexer(inputCol="city", outputCol="city_Index")
stringIndexer = StringIndexer(inputCol="state", outputCol="state_Index")

In [ ]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

df = spark.createDataFrame([
    (0, "a"),
    (1, "b"),
    (2, "c"),
    (3, "a"),
    (4, "a"),
    (5, "c")
], ["id", "category"])

stringIndexer = StringIndexer(inputCol="category", outputCol="categoryIndex")
model = stringIndexer.fit(df)
indexed = model.transform(df)
indexed.show()

encoder = OneHotEncoder(inputCol="categoryIndex", outputCol="categoryVec")
encoded = encoder.transform(indexed)
encoded.show()

In [29]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=["id","categoryVec","categoryVec","categoryVec"],
    outputCol="features")

output = assembler.transform(encoded)
output.show()

+---+--------+-------------+-------------+--------------------+
| id|category|categoryIndex|  categoryVec|            features|
+---+--------+-------------+-------------+--------------------+
|  0|       a|          0.0|(2,[0],[1.0])|(7,[1,3,5],[1.0,1...|
|  1|       b|          2.0|    (2,[],[])|       (7,[0],[1.0])|
|  2|       c|          1.0|(2,[1],[1.0])|[2.0,0.0,1.0,0.0,...|
|  3|       a|          0.0|(2,[0],[1.0])|[3.0,1.0,0.0,1.0,...|
|  4|       a|          0.0|(2,[0],[1.0])|[4.0,1.0,0.0,1.0,...|
|  5|       c|          1.0|(2,[1],[1.0])|[5.0,0.0,1.0,0.0,...|
+---+--------+-------------+-------------+--------------------+



In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

dataset = spark.createDataFrame(
    [(0, 18, 1.0, Vectors.dense([0.0, 10.0, 0.5]), 1.0)],
    ["id", "hour", "mobile", "userFeatures", "clicked"])

assembler = VectorAssembler(
    inputCols=["hour", "mobile", "userFeatures"],
    outputCol="features")

output = assembler.transform(dataset)
print("Assembled columns 'hour', 'mobile', 'userFeatures' to vector column 'features'")
output.select("features", "clicked").show(truncate=False)

In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.mllib.regression import LabeledPoint

# Several of the methods available in scala are currently missing from pyspark
# Load training data in LIBSVM format
data = spark\
    .read.format("libsvm").load("data/mllib/sample_binary_classification_data.txt")\
    .rdd.map(lambda row: LabeledPoint(row[0], row[1]))

# Split data into training (60%) and test (40%)
training, test = data.randomSplit([0.6, 0.4], seed=11)
training.cache()

# Run training algorithm to build the model
model = LogisticRegressionWithLBFGS.train(training)

# Compute raw scores on the test set
predictionAndLabels = test.map(lambda lp: (float(model.predict(lp.features)), lp.label))

# Instantiate metrics object
metrics = BinaryClassificationMetrics(predictionAndLabels)

# Area under precision-recall curve
print("Area under PR = %s" % metrics.areaUnderPR)

# Area under ROC curve
print("Area under ROC = %s" % metrics.areaUnderROC)